In [5]:
import requests
from bs4 import BeautifulSoup
import gzip
import json
from pathlib import Path
from urllib.parse import urljoin
from copy import deepcopy
import random

In [6]:
from typing import Dict, List, Tuple
from collections import defaultdict

DATA_ROOT = Path("data")
CASE_DIRS = ["case14_data", "case57_data", "case118_data", "case300_data"]
MAX_VARIANTS_PER_DAY = 5  # Set to None to disable capping
RNG = random.Random(42)  # Fixed seed for reproducible sampling
OUTPUT_ROOT = DATA_ROOT / "contingency_cases"
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

def load_json(path: Path) -> Dict:
    with open(path, "r") as handle:
        return json.load(handle)

def write_json(path: Path, payload: Dict) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as handle:
        json.dump(payload, handle, indent=2)

def build_contingency_variant(base_payload: Dict, contingency_id: str, affected_lines: List[str]) -> Tuple[Dict, List[str]]:
    """Return a new payload with the specified contingency activated."""
    variant = deepcopy(base_payload)
    missing_lines = []
    lines = variant.get("Transmission lines", {})
    for line_id in affected_lines:
        if lines.pop(line_id, None) is None:
            missing_lines.append(line_id)
    # Remove the activated contingency entry
    contingencies = variant.get("Contingencies", {})
    if contingency_id in contingencies:
        contingencies = {k: v for k, v in contingencies.items() if k != contingency_id}
        variant["Contingencies"] = contingencies
    return variant, missing_lines

def generate_contingency_variants(case_dir: Path) -> Dict[str, int]:
    """Create contingency scenarios for every JSON file in the case directory."""
    produced_counts = defaultdict(int)
    for source_path in sorted(case_dir.glob("*.json")):
        payload = load_json(source_path)
        contingencies = payload.get("Contingencies", {})
        contingency_items = list(contingencies.items())
        if not contingency_items:
            continue
        if MAX_VARIANTS_PER_DAY is not None and len(contingency_items) > MAX_VARIANTS_PER_DAY:
            selected_items = RNG.sample(contingency_items, MAX_VARIANTS_PER_DAY)
        else:
            selected_items = contingency_items
        for contingency_id, contingency_info in selected_items:
            affected_lines = contingency_info.get("Affected lines", [])
            variant, missing_lines = build_contingency_variant(payload, contingency_id, affected_lines)
            if missing_lines:
                print(f"Warning: {source_path.name} missing lines {missing_lines} for contingency {contingency_id}")
            output_dir = OUTPUT_ROOT / case_dir.name
            output_name = f"{source_path.stem}_{contingency_id}.json"
            write_json(output_dir / output_name, variant)
            produced_counts[source_path.stem] += 1
    return produced_counts

def summarize_counts(case_dir: Path, counts: Dict[str, int]) -> None:
    print(f"\nGenerated contingency scenarios for {case_dir.name}:")
    if not counts:
        print("  No contingencies found.")
        return
    for stem, count in counts.items():
        print(f"  {stem}: {count} variants")

overall_counts = {}
for case_name in CASE_DIRS:
    case_path = DATA_ROOT / case_name
    if not case_path.exists():
        print(f"Skipping missing directory: {case_path}")
        continue
    counts = generate_contingency_variants(case_path)
    overall_counts[case_name] = counts
    summarize_counts(case_path, counts)

total_variants = sum(sum(counts.values()) for counts in overall_counts.values())
print(f"\nTotal contingency variants written: {total_variants}")
print(f"Output directory: {OUTPUT_ROOT.resolve()}")


Generated contingency scenarios for case14_data:
  2017-01-01: 5 variants
  2017-01-02: 5 variants
  2017-01-03: 5 variants
  2017-01-04: 5 variants
  2017-01-05: 5 variants
  2017-01-06: 5 variants
  2017-01-07: 5 variants
  2017-01-08: 5 variants
  2017-01-09: 5 variants
  2017-01-10: 5 variants
  2017-01-11: 5 variants
  2017-01-12: 5 variants
  2017-01-13: 5 variants
  2017-01-14: 5 variants
  2017-01-15: 5 variants
  2017-01-16: 5 variants
  2017-01-17: 5 variants
  2017-01-18: 5 variants
  2017-01-19: 5 variants
  2017-01-20: 5 variants
  2017-01-21: 5 variants
  2017-01-22: 5 variants
  2017-01-23: 5 variants
  2017-01-24: 5 variants
  2017-01-25: 5 variants
  2017-01-26: 5 variants
  2017-01-27: 5 variants
  2017-01-28: 5 variants
  2017-01-29: 5 variants
  2017-01-30: 5 variants
  2017-01-31: 5 variants
  2017-02-01: 5 variants
  2017-02-02: 5 variants
  2017-02-03: 5 variants
  2017-02-04: 5 variants
  2017-02-05: 5 variants
  2017-02-06: 5 variants
  2017-02-07: 5 variants
